In [1]:
import nest_asyncio
nest_asyncio.apply()

from llama_index.core.evaluation import (
    DatasetGenerator,
    FaithfulnessEvaluator,
    RelevancyEvaluator
)
from llama_index.llms.openai import OpenAI
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    load_index_from_storage,
    StorageContext,
    get_response_synthesizer
)

In [2]:
filename_fn = lambda filename: {"file_name": filename}
# automatically sets the metadata of each document according to filename_fn
documents = SimpleDirectoryReader(
    input_dir="corpus", file_metadata=filename_fn
).load_data()

In [4]:
from llama_index.core.llama_dataset.generator import RagDatasetGenerator
# llm = OpenAI(model="gpt-3.5-turbo")
# dataset_generator = RagDatasetGenerator.from_documents(
#     documents=documents,
#     llm=llm,
#     num_questions_per_chunk=1,  # set the number of questions per nodes
# )

# rag_dataset = dataset_generator.generate_dataset_from_nodes()
# rag_dataset.save_json("rag_dataset.json")

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-Hi6YdybZb8M18NdArAqb0ZgM on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}

In [5]:
import time

# Define the number of requests allowed per minute and the waiting time between requests
requests_per_minute = 3
wait_time_seconds = 60 / requests_per_minute  # Time to wait between requests

# Initialize the OpenAI model instance
llm = None

# Attempt to create the OpenAI model instance
for _ in range(requests_per_minute):
    try:
        llm = OpenAI(model="gpt-3.5-turbo")
        break  # Exit the loop if the model instance is successfully created
    except Exception as e:
        if "Rate limit reached" in str(e):
            print("Rate limit reached. Waiting before retrying...")
            time.sleep(wait_time_seconds)
        else:
            # Handle other types of exceptions if needed
            print(f"Error: {e}")

# If the model instance is still None after all attempts, handle the error
if llm is None:
    print("Failed to create the OpenAI model instance.")
else:
    # Continue with your code after ensuring the model instance is created
    dataset_generator = RagDatasetGenerator.from_documents(
        documents=documents,
        llm=llm,
        num_questions_per_chunk=1  # set the number of questions per nodes
    )

    rag_dataset = dataset_generator.generate_dataset_from_nodes()
    rag_dataset.save_json("rag_dataset.json")


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-Hi6YdybZb8M18NdArAqb0ZgM on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}